In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import re
import matplotlib.ticker as mtick
import pandas as pd
import scienceplots
plt.style.use('science')

In [2]:
activations = ['sigmoid', 'tanh', 'relu', 'JNB1', 'JNB2', 'JNB3', 'JNB4', 'JNB5', 'JNB6']

DATA = []
SCORES = []
for activation in activations:
    files = sorted(glob.glob('data/*%s*.txt' % activation))
    data = []
    score = []
    for file in files:
        history, score_ = list(map(lambda t: eval(str.strip(t)),open(file).readlines()))

        historyold = history.copy()
        history={}
        for key in historyold.keys():

            if key[-2] == '_':

                history[key[:-2]] = historyold[key]
            else:
                history[key] = historyold[key]
        
        d = history['loss'],history['val_loss'],history['accuracy'],history['val_accuracy'], history['precision'],history['val_precision'],history['recall'],history['val_recall'], history['auc'],history['val_auc']
        data.append(d)
        score.append(score_)
    DATA.append(data)
    SCORES.append(score)
DATA = np.array(DATA)
SCORES=np.array(SCORES)
DATA.shape, SCORES.shape

((9, 5, 10, 10), (9, 5, 5))

In [3]:
log = lambda x: x # np.log(x)/np.log(50)
DATA[:,:,2:,:] = log(DATA[:,:,2:,:])
mean = (DATA).mean(axis=1)

var = (DATA).var(axis=1)

mean.shape

(9, 10, 10)

In [4]:
mapping = {'JNB1':'JNB(1)', 'JNB2':'JNB(2)', 'JNB3':'JNB(3)', 'JNB4':'JNB(4)', 
           'JNB5':'JNB(5)', 'JNB6':'JNB(6)', 
           'RELU':'ReLU', 'SIGMOID':'Sigmoid','TANH':'Tanh'}


names = []
for i in range(0,len(activations)):
    names.append(mapping[activations[i].upper()])

In [5]:
SCORES[:, :, 1:] *= 100
mean = SCORES.mean(axis=1)
std =  SCORES.std(axis=1)

In [6]:
columns = ['Loss', 'Accuracy','PR','RC','AUC']
df = pd.DataFrame(index=names, columns=columns)

# Populate DataFrame with mean +- std
for i in range(mean.shape[0]):
    for j in range(mean.shape[1]):
        df.at[names[i], columns[j]] = f"${mean[i, j]:.3f} \pm {std[i, j]:.3f}$"

df

,Loss,Accuracy,PR,RC,AUC
Sigmoid,$0.604 \pm 0.039$,$85.198 \pm 0.468$,$81.440 \pm 1.511$,$91.266 \pm 1.532$,$91.631 \pm 0.422$
Tanh,$0.610 \pm 0.060$,$85.604 \pm 0.818$,$82.364 \pm 2.790$,$90.912 \pm 2.705$,$91.894 \pm 0.505$
ReLU,$0.808 \pm 0.064$,$83.826 \pm 1.141$,$78.197 \pm 2.124$,$93.995 \pm 1.358$,$91.080 \pm 0.404$
JNB(1),$0.724 \pm 0.074$,$84.462 \pm 1.156$,$79.967 \pm 3.160$,$92.387 \pm 3.090$,$91.241 \pm 0.501$
JNB(2),$0.671 \pm 0.044$,$85.205 \pm 0.591$,$80.559 \pm 1.374$,$92.882 \pm 1.126$,$91.306 \pm 0.478$
JNB(3),$0.646 \pm 0.057$,$84.828 \pm 0.761$,$80.100 \pm 1.787$,$92.808 \pm 1.404$,$91.576 \pm 0.416$
JNB(4),$0.559 \pm 0.062$,$85.645 \pm 0.762$,$83.395 \pm 3.549$,$89.498 \pm 3.916$,$92.497 \pm 0.404$
JNB(5),$0.691 \pm 0.100$,$84.247 \pm 1.479$,$79.283 \pm 3.401$,$93.190 \pm 2.584$,$91.454 \pm 0.825$
JNB(6),$0.627 \pm 0.107$,$84.455 \pm 1.658$,$80.079 \pm 4.017$,$92.371 \pm 3.215$,$91.648 \pm 0.985$


In [7]:
df = pd.DataFrame({
    'Loss mean': mean[:, 0],
    'Loss std': std[:, 0],
    'Accuracy mean': mean[:, 1],
    'Accuracy std': std[:, 1],
}, index = names)
df.round(4)

,Loss mean,Loss std,Accuracy mean,Accuracy std
Sigmoid,0.6036,0.0394,85.1976,0.4682
Tanh,0.6098,0.0596,85.6040,0.8180
ReLU,0.8077,0.0640,83.8264,1.1412
JNB(1),0.7241,0.0735,84.4624,1.1555
JNB(2),0.6711,0.0441,85.2048,0.5910
JNB(3),0.6462,0.0573,84.8280,0.7613
JNB(4),0.5591,0.0623,85.6448,0.7619
JNB(5),0.6912,0.0995,84.2472,1.4793
JNB(6),0.6268,0.1070,84.4552,1.6579
